In [2]:
import os
import json

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [11]:
# obtain skeletons + label

DSET_PATH = "/Users/steventan/.cache/kagglehub/datasets/soumicksarker/ipn-hand-dataset/versions/7"
TRAIN_PATH = os.path.join(DSET_PATH, "train_skeletons")
TEST_PATH = os.path.join(DSET_PATH, "test_skeletons")

def load_skel_data(path):
    skeletons = torch.load(os.path.join(path, "skeletons_tensor.pt"))  
    with open(os.path.join(path, "skeleton_annots.json"), "r") as f:
        metadata = json.load(f)
    return skeletons, metadata

train_skels, train_metadata = load_skel_data(TRAIN_PATH)
test_skels, test_metadata = load_skel_data(TEST_PATH)

train_labels = torch.tensor(
    [sample["label_id"] for sample in train_metadata["samples"]],
    dtype=torch.long,
)
test_labels = torch.tensor(
    [sample["label_id"] for sample in test_metadata["samples"]],
    dtype=torch.long,
)


# filter out first 100 training datapoints since they aren't labelled well
train_skels = train_skels[100:]
train_labels = train_labels[100:]

num_classes = len(torch.unique(train_labels)) + 1

# sanity check stuff
print("Labels shape:", train_labels.shape)
print("Skeletons shape:", train_skels.shape)
print("Num classes:", n_classes)

Labels shape: torch.Size([3781])
Skeletons shape: torch.Size([3781, 210, 21, 3])
Num classes: 14


/var/folders/_1/ph20kymj4s9bnnn49x3mjlxr0000gn/T/ipykernel_31393/3354176845.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  skeletons = torch.load(os.path.join(path, "sk

In [12]:
# training stuff 
train_ds = TensorDataset(train_skels, train_labels)
test_ds = TensorDataset(test_skels, test_labels)

batch_size = 32
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

print("Train samples:", len(train_ds))
print("Test samples:", len(test_ds))

Train samples: 3781
Test samples: 1556


In [13]:
T, J, C = train_skels.shape[1:]  
input_dim = T * J * C

class BasicGLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 420),
            nn.ReLU(),
            nn.Linear(420, 67),
            nn.ReLU(),
            nn.Linear(67, num_classes),
        )

    def forward(self, x):
        x = x.view(x.size(0), -1) 
        return self.net(x)

device = torch.device("cpu")
model = BasicGLP(input_dim, num_classes).to(device)
print(model)

BasicGLP(
  (net): Sequential(
    (0): Linear(in_features=13230, out_features=420, bias=True)
    (1): ReLU()
    (2): Linear(in_features=420, out_features=67, bias=True)
    (3): ReLU()
    (4): Linear(in_features=67, out_features=15, bias=True)
  )
)


In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# training loop
def run_epoch(loader, model, criterion, optimizer=None):
    if optimizer is None:
        model.eval()
    else:
        model.train()

    total_loss = 0.0
    correct = 0
    total = 0

    for x_batch, y_batch in loader:
        x_batch = x_batch.to(device, dtype=torch.float32)
        y_batch = y_batch.to(device)

        if optimizer is not None:
            optimizer.zero_grad()

        logits = model(x_batch)           # forward pass
        loss = criterion(logits, y_batch) # compute loss

        if optimizer is not None:
            loss.backward()               # backprop
            optimizer.step()              # update weights

        total_loss += loss.item() * x_batch.size(0)

        preds = logits.argmax(dim=1)
        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)

    avg_loss = total_loss / total
    acc = correct / total
    return avg_loss, acc

num_epochs = 15

for epoch in range(1, num_epochs + 1):
    train_loss, train_acc = run_epoch(train_loader, model, criterion, optimizer)
    test_loss, test_acc = run_epoch(test_loader, model, criterion, optimizer=None)

    print(
        f"Epoch {epoch:02d} | "
        f"train_loss={train_loss:.4f}, train_acc={train_acc:.3f} | "
        f"test_loss={test_loss:.4f}, test_acc={test_acc:.3f}"
    )

Epoch 01 | train_loss=2.1838, train_acc=0.325 | test_loss=2.0318, test_acc=0.410
Epoch 02 | train_loss=1.8169, train_acc=0.454 | test_loss=1.6637, test_acc=0.540
Epoch 03 | train_loss=1.6006, train_acc=0.507 | test_loss=1.6182, test_acc=0.518
Epoch 04 | train_loss=1.4148, train_acc=0.553 | test_loss=1.3920, test_acc=0.564
Epoch 05 | train_loss=1.3067, train_acc=0.582 | test_loss=1.3255, test_acc=0.600
Epoch 06 | train_loss=1.1230, train_acc=0.641 | test_loss=1.4665, test_acc=0.551
Epoch 07 | train_loss=1.0177, train_acc=0.661 | test_loss=1.1375, test_acc=0.646
Epoch 08 | train_loss=0.9837, train_acc=0.675 | test_loss=1.1002, test_acc=0.655
Epoch 09 | train_loss=0.8596, train_acc=0.709 | test_loss=1.0120, test_acc=0.680
Epoch 10 | train_loss=0.8750, train_acc=0.703 | test_loss=1.2160, test_acc=0.617
Epoch 11 | train_loss=0.8607, train_acc=0.697 | test_loss=1.1193, test_acc=0.654
Epoch 12 | train_loss=0.7809, train_acc=0.733 | test_loss=1.1084, test_acc=0.651
Epoch 13 | train_loss=0.8007